In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

In [8]:
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# 학습에 필요한 설정값들 정의
learning_rate_rmsprop=0.02
learning_rate_gradient=0.5

num_epochs=100
batch_size=256
display_step=1
input_size=784
hidden1_size=128
hidden2_size=64

In [5]:
# 입력받기 위한 값 설정
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

In [6]:
# actoencoder 설정
def build_autoencoder(x):
    #784->128 피쳐를 줄이면서 중요한 피쳐들을 뽑아내는 과정
    w1=tf.Variable(tf.random_normal([input_size,hidden1_size]))
    b1=tf.Variable(tf.random_normal([hidden1_size]))
    L1=tf.nn.sigmoid(tf.matmul(x,w1)+b1)
    #128->64
    w2=tf.Variable(tf.random_normal([hidden1_size,hidden2_size]))
    b2=tf.Variable(tf.random_normal([hidden2_size]))
    L2=tf.nn.sigmoid(tf.matmul(L1,w2)+b2)
    #64->128 뽑아낸 피쳐들로부터 다시 원래 데이터를 다시 재구축하는 과정
    w3=tf.Variable(tf.random_normal([hidden2_size,hidden1_size]))
    b3=tf.Variable(tf.random_normal([hidden1_size]))
    L3=tf.nn.sigmoid(tf.matmul(L2,w3)+b3)
    #128->784 
    w4=tf.Variable(tf.random_normal([hidden1_size,input_size]))
    b4=tf.Variable(tf.random_normal([input_size]))
    reconstructed_x=tf.nn.sigmoid(tf.matmul(L3,w4)+b4)
    
    return reconstructed_x, L2    

In [11]:
# sofrmax_분류기
def build_softmax_classify(x):
    w_softmax=tf.Variable(tf.zeros([hidden2_size,10])) #원본이미지를 오토인코더로 최대한 압축해서 얻은 피쳐들을 다시 10차원으로
    b_softmax=tf.Variable(tf.zeros([10]))
    y_pred=tf.nn.softmax(tf.matmul(x,w_softmax)+b_softmax)
    
    return y_pred

In [12]:
# 오토 인코더를 분류된 예측값, softmax로 표시된 예측값
y_pred_autoencoder,extracted_feature=build_autoencoder(x)
y_true_autoencoder=x  #오토 인코더의 타깃값은 들어온 x다 그래서 unsupervised가 가능
y_pred_softmax=build_softmax_classify(extracted_feature)

In [13]:
# 1. pre-training: mnist 데이터 재구축을 목적으로하는 손실함수와 옵티마이저를 정의한다.
pretraining_loss=tf.reduce_mean(tf.pow(y_true_autoencoder-y_pred_autoencoder,2)) # squared error 함수
pretraining_train_step=tf.train.RMSPropOptimizer(learning_rate_rmsprop).minimize(pretraining_loss)

# 2. fine-tuning: minist 데이터 분류를 목적으로 하는 손실함수와 옵티마이저를 구축한다.
finetuning_loss=tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_pred_softmax),reduction_indices=[1]))
finetuning_train_step=tf.train.GradientDescentOptimizer(learning_rate_gradient).minimize(finetuning_loss)


In [17]:
# 셰션을 열고 그래프를 실행

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch=int(mnist.train.num_examples/batch_size)
    # 1. minst 데이터 재구축을 위한 오토인코더 최적화(pre-training)
    for epoch in range(num_epochs):
        
        for i in range(total_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            _,pretraining_loss_print=sess.run([pretraining_train_step,pretraining_loss],feed_dict={x:batch_xs,y:batch_ys})
            
        if epoch % display_step ==0:
            print("반복(epoch):%d, pretrain_loss(손실): %f" % ((epoch+1),pretraining_loss_print))
    print("step1: MNIST 데이터 재구축을 위한 오토인코더 최적화 완료")
    
    # 2. mnist 데이터 분류를 위한 오토인코더 + softmax 분류기 최적화(fine_tuning)
    for epoch in range(num_epochs+100):
        
        for i in range(total_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            _,finetuning_loss_print=sess.run([finetuning_train_step,finetuning_loss],feed_dict={x:batch_xs,y:batch_ys})
        
        print("반복(epoch):%d, fine_tuning_loss(손실):%f" % ((epoch+1),finetuning_loss_print))
        
    print("step2: MNIST 데이터 분류를 위한 오토인코더 + 소프트 맥스 분류기 최적화 완료")
    
    
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_pred_softmax,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print("정확도:%f" % sess.run(accuracy,feed_dict={x:mnist.test.images, y:mnist.test.labels}))

반복(epoch):1, pretrain_loss(손실): 0.167531
반복(epoch):2, pretrain_loss(손실): 0.116206
반복(epoch):3, pretrain_loss(손실): 0.100494
반복(epoch):4, pretrain_loss(손실): 0.090414
반복(epoch):5, pretrain_loss(손실): 0.084163
반복(epoch):6, pretrain_loss(손실): 0.077649
반복(epoch):7, pretrain_loss(손실): 0.077102
반복(epoch):8, pretrain_loss(손실): 0.071776
반복(epoch):9, pretrain_loss(손실): 0.070340
반복(epoch):10, pretrain_loss(손실): 0.068978
반복(epoch):11, pretrain_loss(손실): 0.064038
반복(epoch):12, pretrain_loss(손실): 0.057467
반복(epoch):13, pretrain_loss(손실): 0.056387
반복(epoch):14, pretrain_loss(손실): 0.048961
반복(epoch):15, pretrain_loss(손실): 0.050635
반복(epoch):16, pretrain_loss(손실): 0.047107
반복(epoch):17, pretrain_loss(손실): 0.044837
반복(epoch):18, pretrain_loss(손실): 0.045899
반복(epoch):19, pretrain_loss(손실): 0.043379
반복(epoch):20, pretrain_loss(손실): 0.047415
반복(epoch):21, pretrain_loss(손실): 0.042141
반복(epoch):22, pretrain_loss(손실): 0.041209
반복(epoch):23, pretrain_loss(손실): 0.040105
반복(epoch):24, pretrain_loss(손실): 0.039625
반

반복(epoch):92, fine_tuning_loss(손실):0.032250
반복(epoch):93, fine_tuning_loss(손실):0.036786
반복(epoch):94, fine_tuning_loss(손실):0.038612
반복(epoch):95, fine_tuning_loss(손실):0.037038
반복(epoch):96, fine_tuning_loss(손실):0.026153
반복(epoch):97, fine_tuning_loss(손실):0.038301
반복(epoch):98, fine_tuning_loss(손실):0.084209
반복(epoch):99, fine_tuning_loss(손실):0.029764
반복(epoch):100, fine_tuning_loss(손실):0.047113
반복(epoch):101, fine_tuning_loss(손실):0.033317
반복(epoch):102, fine_tuning_loss(손실):0.033871
반복(epoch):103, fine_tuning_loss(손실):0.035705
반복(epoch):104, fine_tuning_loss(손실):0.047846
반복(epoch):105, fine_tuning_loss(손실):0.018035
반복(epoch):106, fine_tuning_loss(손실):0.027655
반복(epoch):107, fine_tuning_loss(손실):0.034073
반복(epoch):108, fine_tuning_loss(손실):0.056265
반복(epoch):109, fine_tuning_loss(손실):0.021892
반복(epoch):110, fine_tuning_loss(손실):0.032974
반복(epoch):111, fine_tuning_loss(손실):0.018196
반복(epoch):112, fine_tuning_loss(손실):0.045273
반복(epoch):113, fine_tuning_loss(손실):0.026624
반복(epoch):114, fin

In [ ]:
%pylab inline
import numpy as np

# 임의로 선택한 10개의 MNIST 데이터를 화면에 보여준다.
for c in range(1, 11):
    subplot(2, 5, c)
    i = randint(mnist.test.num_examples)
    im = np.reshape(mnist.test.images[i], [28, 28])
    axis("off")
    imshow(im, cmap='gray')

In [ ]:
f,a=plt.subplots(2,10,figsize=(10,2))
    for i in range(example_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i],(28,28)))
        a[1][i].imshow(np.reshape(reconstucted_result[i],(28,28)))
                       
    f.show()
    plt.draw()
    plt.waitforbuttonpress()